This are the results of group 42. 
- We first load the data with a 90:10 distribution
  - 90% of the data is used for training
  - 10% of the data is used for testing
- We find the smallest image dimensions by searching for the smallest shapes within 
- We used the architecture from the tensorflow tutorial.
- At the end we reach an accuracy of 95 %

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras as keras

ModuleNotFoundError: No module named 'tensorflow'

# 1 Data set

In [ ]:
# Load first 90% as training data and last 10% as test data
ds_train, ds_info = tfds.load("malaria",split="train[:90%]", as_supervised=True, shuffle_files=True, with_info=True)
ds_valid = tfds.load("malaria",split="train[90%:100%]", as_supervised=True, shuffle_files=True)

In [ ]:
# find the smallest shapes of the images
def findMinMax(ds1, ds2):
    minX1 = min(i[0].shape[0] for i in ds1)
    minY1 = min(i[0].shape[1] for i in ds1)

    minX2 = min(i[0].shape[0] for i in ds2)
    minY2 = min(i[0].shape[1] for i in ds2)
    return min(minX1, minX2), min(minY1, minY2)

# save the smallest shapes of the dataset
minX, minY = findMinMax(ds_train, ds_valid)
print(minX, minY)

In [ ]:
def normalize_img(image, label):
  # Resize the images to the smallest shape
  image = tf.image.resize(image, [minY, minX])
  # normalize the images and transfer labels to one hot vectors
  return tf.cast(image, tf.float32) / 255., tf.one_hot(label,2)

In [ ]:
# Inspect dataset
fig = tfds.show_examples(ds_train, ds_info)

# normalize the data and turn it into floats
ds_train = ds_train.map(normalize_img)
# Save data in the cache
ds_train = ds_train.cache() # not very usefull since data is very small
# Shuffle the whole training dataset
ds_train = ds_train.shuffle(round(ds_info.splits['train'].num_examples*0.9))
# Make minibatches to improve the direction of the gradient
ds_train = ds_train.batch(64)
# allow prefetching to gain better performance
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

# Repeat steps for test dataset (except shuffle)
ds_valid = ds_valid.map(normalize_img)
ds_valid = ds_valid.cache() # not very usefull since data is very small
ds_valid = ds_valid.batch(64)
ds_valid = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

# 2 Model
CNN Architecture is from [tensorflow tutorial](https://www.tensorflow.org/tutorials/images/cnn)

In [ ]:
model = keras.Sequential([

    # First Convolutional Block
    keras.layers.Conv2D(filters=32, kernel_size=3, activation="relu", input_shape=(46, 40, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    # Second Convolutional Block
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    # Third Convolutional Layer
    keras.layers.Conv2D(64, (3, 3), activation='relu'),

    # Classifier Head
    keras.layers.Flatten(),
    keras.layers.Dense(units=10, activation="relu"),
    keras.layers.Dense(units=2, activation="softmax"),
])
# print architecture of the model
model.summary()

# 3 Training

In [ ]:
# Execute the training with the Adam optimizer
# as discussed in the lecture
model.compile(
    optimizer=tf.keras.optimizers.Adam(epsilon=0.01),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# train the model 15 epochs
history = model.fit(
    ds_train,
    validation_data=ds_valid,
    epochs=15,
)


# 4 Visualization

In [3]:
# ploting the resulting data
x_axis = np.arange(1, 16)

# somehow test accuracy is very case sensitive
fig, ax = plt.subplots(1,2)
ax[0].plot(x_axis, history.history["loss"])
ax[0].plot(x_axis, history.history["val_loss"])
ax[1].plot(x_axis, history.history["accuracy"])
ax[1].plot(x_axis, history.history["val_accuracy"])


NameError: name 'np' is not defined